In [1]:
import os
import sys

import django

sys.path.append('../')  # add path to project root dir

os.environ["DJANGO_SETTINGS_MODULE"] = "flourish.settings"

os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

django.setup()

  * Reading config from flourish.ini
Loading Data Encryption (init)...
 * loading keys from /Users/nimrodmunatsi/source/flourish-code-space/flourish/crypto_fields
 * loading rsa.restricted.public ... Done.
 * loading rsa.restricted.private ... Done.
 * loading rsa.local.public ... Done.
 * loading rsa.local.private ... Done.
 * loading aes.local ... Done.
 * loading aes.restricted ... Done.
 * loading salt.local ... Done.
 * loading salt.restricted ... Done.
 Done loading Data Encryption (init)...


/Users/nimrodmunatsi/.venv/flourish/lib/python3.9/site-packages/django_q/conf.py:139: UserWarning: Retry and timeout are misconfigured. Set retry larger than timeout, 
        failure to do so will cause the tasks to be retriggered before completion. 
        See https://django-q.readthedocs.io/en/latest/configure.html#retry for details.
  warn(
/Users/nimrodmunatsi/.venv/flourish/lib/python3.9/site-packages/simple_history/models.py:164: UserWarning: HistoricalRecords added to abstract model (HistoryManagerMixin) without inherit=True
  warnings.warn(msg, UserWarning)
/Users/nimrodmunatsi/.venv/flourish/lib/python3.9/site-packages/simple_history/models.py:164: UserWarning: HistoricalRecords added to abstract model (ScheduleModelMixin) without inherit=True
  warnings.warn(msg, UserWarning)
/Users/nimrodmunatsi/.venv/flourish/lib/python3.9/site-packages/simple_history/models.py:164: UserWarning: HistoricalRecords added to abstract model (VisitModelMixin) without inherit=True
  warnings.wa

Loading Data Encryption ...
 * found encryption keys in /Users/nimrodmunatsi/source/flourish-code-space/flourish/crypto_fields.
 * using model django_crypto_fields.crypt.
 Done loading Data Encryption.
Loading Edc Consent ...
 * checking for site consents ...
 * registered consents 'consents' from 'flourish_caregiver'
 * registered consents 'consents' from 'flourish_child'
 * registered consents 'consents' from 'pre_flourish'
 * registered consents 'consents' from 'flourish_facet'
 * flourish_caregiver.subjectconsent 1 covering 2020-07-01 UTC to 2025-06-30 UTC
 * flourish_caregiver.subjectconsent 2 covering 2020-07-01 UTC to 2025-06-30 UTC
 * flourish_caregiver.subjectconsent 3 covering 2020-07-01 UTC to 2025-06-30 UTC
 * flourish_caregiver.subjectconsent 4 covering 2020-07-01 UTC to 2025-06-30 UTC
 * flourish_caregiver.tbadolconsent 1 covering 2020-07-01 UTC to 2025-06-30 UTC
 * flourish_caregiver.tbinformedconsent 1 covering 2020-07-01 UTC to 2025-06-30 UTC
 * flourish_child.childdum

In [2]:
import json
from datetime import datetime
import pytz
from tqdm import tqdm
from edc_constants.constants import NO
from edc_base.utils import age
from flourish_caregiver.models import CaregiverChildConsent
from flourish_child.models import ChildVisit, InfantHIVTesting18Months, \
    InfantHIVTesting9Months, \
    InfantHIVTestingAge6To8Weeks, InfantHIVTestingBirth, InfantHIVTestingOther


def read_json_file(file_path):
    with open(file_path, 'r') as json_file:
        data = json.load(json_file)
    return data

def fetch_child_consent(subject_identifier):
    return CaregiverChildConsent.objects.filter(subject_identifier=subject_identifier).latest('consent_datetime')

def fetch_child_visit(subject_identifier, visit):
    return ChildVisit.objects.get(visit_code=visit[3],
                                  subject_identifier=subject_identifier,
                                  visit_code_sequence=visit[4])
def age_at_test_in_weeks(child_dob, report_datetime):
    age_at_test = age(child_dob, report_datetime)
    return (age_at_test.years * 52.143) + (age_at_test.months * 4.348)

def create_or_update_infant_hiv_testing(test_obj_class, options):
    try:
        test_obj = test_obj_class.objects.get(child_visit=options['child_visit'])
        for attr, value in options.items():
            setattr(test_obj, attr, value)
        test_obj.save()
    except test_obj_class.DoesNotExist:
        test_obj = test_obj_class.objects.create(**options)
    return test_obj

def read_and_update_infant_hiv_tests():
    infant_hiv_test_objs = read_json_file(
        '/Users/nimrodmunatsi/Downloads/infanthivtesting.json')

    for infant_hiv_test_obj in tqdm(infant_hiv_test_objs):
        if infant_hiv_test_obj.get('fields').get(
                'child_tested_for_hiv') in [NO, 'Dont_know']:
            continue

        visit = infant_hiv_test_obj.get('fields').get('child_visit')
        subject_identifier = visit[0]
        child_consent = fetch_child_consent(subject_identifier)
        if not (child_consent.child_dob and infant_hiv_test_obj.get('fields').get(
                'child_test_date')):
            continue

        report_datetime = datetime.strptime(infant_hiv_test_obj.get('fields').get('report_datetime'), "%Y-%m-%dT%H:%M:%SZ").replace(tzinfo=pytz.UTC)

        try:
            age_at_test_weeks = age_at_test_in_weeks(child_consent.child_dob, report_datetime)
        except ValueError:
            print(f'Child {subject_identifier}\'s dob {child_consent.child_dob} '
                  f'is less than the test date {report_datetime}')
            continue

        try:
            child_visit = fetch_child_visit(subject_identifier, visit)
        except ChildVisit.DoesNotExist:
            print(f'Child visit {visit[3]} does not exist for {subject_identifier}')
            continue

        base_fields = infant_hiv_test_obj.get('fields')

        options = {
            'child_visit': child_visit,
            'report_datetime': child_visit.report_datetime,
            'child_tested_for_hiv': base_fields.get('child_test_date'),
            'child_test_date_estimated': base_fields.get('child_test_date_estimated'),
            'results_received': base_fields.get('results_received'),
            'recall_result_date': base_fields.get('recall_result_date'),
            'received_date': base_fields.get('received_date'),
            'result_date_estimated': base_fields.get('result_date_estimated'),
            'hiv_test_result': base_fields.get('hiv_test_result'),
            'additional_comments': base_fields.get('additional_comments'),
        }

        if 6 <= age_at_test_weeks <= 8:
            create_or_update_infant_hiv_testing(InfantHIVTestingAge6To8Weeks, options)
        elif age_at_test_weeks >= 9 * 4.348 and age_at_test_weeks < 10 * 4.348:  # 9 - 10 months
            create_or_update_infant_hiv_testing(InfantHIVTesting9Months, options)
        elif age_at_test_weeks >= 18 * 4.348 and age_at_test_weeks < 19 * 4.348:  # 18 months
            create_or_update_infant_hiv_testing(InfantHIVTesting18Months, options)
        elif age_at_test_weeks == 0:
            create_or_update_infant_hiv_testing(InfantHIVTestingBirth, options)
        else:
            create_or_update_infant_hiv_testing(InfantHIVTestingOther, options)
            
            
read_and_update_infant_hiv_tests()      


  0%|          | 0/265 [00:00<?, ?it/s]

Child visit 3001 does not exist for B142-040990558-4-60
Child visit 2007 does not exist for B142-040990608-7-10


  6%|▋         | 17/265 [00:05<01:02,  4.00it/s]

Child visit 2003 does not exist for B142-040991053-5-10


 11%|█         | 29/265 [00:10<01:34,  2.51it/s]

...

 83%|████████▎ | 219/265 [01:08<00:23,  1.99it/s]

Child visit 2001 does not exist for B142-040991156-6-10


 91%|█████████ | 241/265 [01:13<00:07,  3.26it/s]

Child visit 2001 does not exist for B142-040991130-1-10


100%|██████████| 265/265 [01:18<00:00,  3.36it/s]


In [14]:
dir(ChildVisit)

['ADMIN_SITE_NAME',
 'DoesNotExist',
 'Meta',
 'MultipleObjectsReturned',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_check_column_name_clashes',
 '_check_constraints',
 '_check_field_name_clashes',
 '_check_fields',
 '_check_id_field',
 '_check_index_together',
 '_check_indexes',
 '_check_local_fields',
 '_check_long_column_names',
 '_check_m2m_through_same_relationship',
 '_check_managers',
 '_check_model',
 '_check_model_name_db_lookup_clashes',
 '_check_ordering',
 '_check_property_name_related_field_accessor_clashes',
 '_check_single_primary_key',
 '_check_swappable',
 '_check_unique_together',
 '_check_visit_reason_keys',
 